In [ ]:
import requests
import pandas as pd
import mplfinance as mpf 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec


# Construir la URL de la API
url ='https://fapi.binance.com/fapi/v1/klines'

params = {
    'symbol' : 'btcusdt', # Símbolo del par de negociación
    'interval' : '1d', # Intervalo de tiempo (ejemplo: 1d para datos diarios)
    'limit' : '300' # Límite de barras de kline/candlestick a obtener (máximo 1500)
}   

# Realizar la solicitud GET a la API
response = requests.get(url,params=params)

data = response.json()
data = pd.DataFrame(data)

#Cambio de columnas de nombres
data = data.rename(columns={1:'Open', 2:'High', 3:'Low', 4:'Close', 5:'Volume', 6 : 'timestamp', 7:'Quote_asset_volume', 8 :'Number_of_trades', 9 : 'Taker_buy_base_asset_volume', 10: 'Taker_buy_quote_asset_volume', 11: 'Ignore'})

# Convertir la columna 'datetime' que tiene el formato timestamp a tipo datetime y dejar solo la fecha
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms').dt.date

# Establecer la columna 'fecha' como índice
data.set_index('timestamp', inplace=True)
data.index = pd.to_datetime(data.index)

#Eliminar una columna
data = data.drop(0, axis=1)
data = data.drop('Ignore', axis=1)

# Convertir las columnas a tipo de dato float64
data = data.astype('Float64')

# Seleccionar solo algunas columnas para graficar
data_ohlc = data[['Open', 'High', 'Low', 'Close']]
data_ohlc.index = pd.to_datetime(data_ohlc.index)

#------Datos Adicionales----
data_volumenes = data[['Volume','Quote_asset_volume','Taker_buy_base_asset_volume','Taker_buy_quote_asset_volume','Number_of_trades']]
data_volumenes.index = pd.to_datetime(data_volumenes.index)

#-------Graficar------

# Crear una figura con una cuadrícula de 2x1 con diferentes alturas para las subtramas
fig = plt.figure(figsize=(16, 13))
plt.suptitle("Gráfico de Datos de Binance API - Visualización de Información Completa ", y = 0.9)
gs = gridspec.GridSpec(6, 1, height_ratios=[3, 1, 1, 1, 1, 1])  # Altura del segundo gráfico es la mitad del primero

# Definir la primera subtrama con la altura del 75% de la figura
ax1 = plt.subplot(gs[0])
mpf.plot(data_ohlc, type='candle', figratio=(10,14), style="yahoo", ax=ax1)


# Ajustar atributos del eje x para mostrar las fechas de forma horizontal
ax1.xaxis.set_tick_params(rotation=0)  # Sin rotación


ax1.legend(["BTC"], loc='lower left', handlelength=0)

# Lógica para determinar el color de las barras
colors = ['g' if data_ohlc['Close'][i] > data_ohlc['Open'][i] else 'r' for i in range(len(data_ohlc))]

# Definir la segunda subtrama con la altura del 25% de la figura
ax2 = plt.subplot(gs[1])
ax2.bar(data_volumenes.index, data_volumenes['Volume'], lw=1.5, color= colors)
ax2.set_xlabel('Volumen')


# Definir la tercer subtrama con la altura del 25% de la figura
ax3 = plt.subplot(gs[2])
ax3.bar(data_volumenes.index, data_volumenes['Quote_asset_volume'], lw=1.5, color=colors)
ax3.set_xlabel('Volumen del activo de cotización')


# Definir la cuarta subtrama 
ax4 = plt.subplot(gs[3])
ax4.bar(data_volumenes.index, data_volumenes['Taker_buy_base_asset_volume'], lw=1.5, color=colors)
ax4.set_xlabel(' Volumen del activo base comprado por tomadores')


# Definir la quinta subtrama 
ax5 = plt.subplot(gs[4])
ax5.bar(data_volumenes.index, data_volumenes['Taker_buy_quote_asset_volume'], lw=1.5, color=colors)
ax5.set_xlabel('Volumen del activo de cotización comprado por tomadores')


# Definir la sexta subtrama
ax6 = plt.subplot(gs[5])
ax6.bar(data_volumenes.index, data_volumenes['Number_of_trades'], lw=1.5, color=colors)
ax6.set_xlabel('Número de operaciones')


# Ajustar el espacio entre las subtramas verticalmente
plt.subplots_adjust(hspace=0.7)

# Mostrar el gráfico
plt.show()